In [1]:
import pandas as pd
import numpy as np
from ta.volatility import AverageTrueRange
from tqdm import tqdm

In [2]:
df_daily = pd.read_csv("../src/daily_data/Si_230101_240930.csv", sep=";")
df_min = pd.read_excel("../src/stock_data.xlsx", sheet_name="Si")

In [3]:
def prepare_df(df: pd.DataFrame):
    df = df.rename(columns={col: col.replace("<", "").replace(">", "") for col in df.columns})
    df.drop(["TICKER", "PER"], axis=1, inplace=True)
    df["DATE"] = pd.to_datetime(df["DATE"], format="%y%m%d")
    return df

In [4]:
def get_atr(data, window=14):
    atr = AverageTrueRange(
        high=data["HIGH"],
        low=data["LOW"],
        close=data["CLOSE"],
        window=window
    ).average_true_range()
    return atr

In [5]:
df_daily = prepare_df(df_daily)
df_min = prepare_df(df_min)

In [6]:
myatr = get_atr(df_daily)
df_daily["ATR"] = round(myatr, 2)

In [7]:
df_daily.head(3)

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,ATR
0,2023-01-03,0,70105,72280,69873,72236,920700,0.0
1,2023-01-04,0,72227,72600,71310,72530,846368,0.0
2,2023-01-05,0,72510,72844,71675,72563,708928,0.0


In [8]:
df_daily = df_daily.loc[df_daily["ATR"] > 0]

In [9]:
df_daily["CANDLE_COLOR"] = np.where(df_daily["CLOSE"] > df_daily["OPEN"], 1, 0)
df_daily["PREV_CANDLE_COLOR"] = df_daily["CANDLE_COLOR"].shift()

In [10]:
df_daily["SL_SIZE"] = round(df_daily["ATR"] / 3, 0)
df_daily["TP_SIZE"] = round(df_daily["ATR"] * 2 / 3, 0)
# df_daily["LONG_SL"] = df_daily["CLOSE"] - df_daily["SL_SIZE"]
# df_daily["LONG_TP"] = df_daily["CLOSE"] + df_daily["TP_SIZE"]
# df_daily["SHORT_SL"] = df_daily["CLOSE"] + df_daily["SL_SIZE"]
# df_daily["SHORT_TP"] = df_daily["CLOSE"] - df_daily["TP_SIZE"]

for col in ["SL_SIZE", "TP_SIZE"]:
    df_daily[col] = df_daily[col].shift()

In [11]:
df_daily = df_daily.dropna()

In [12]:
df_min = df_min.merge(df_daily[[
    "DATE",
    "ATR",
    "PREV_CANDLE_COLOR",
    "SL_SIZE",
    "TP_SIZE",
    # "LONG_SL",
    # "LONG_TP",
    # "SHORT_SL",
    # "SHORT_TP"
]], how="left", on="DATE")

In [13]:
df_min = df_min.dropna()

In [14]:
df_daily["PROFIT_LONG"] = 0
df_daily["PROFIT_SHORT"] = 0

In [15]:
df_daily = df_daily.loc[df_daily["DATE"].isin(df_min["DATE"].unique())]
df_min = df_min.loc[df_min["DATE"].isin(df_daily["DATE"].unique())]

In [16]:
df_min.head()

,DATETIME_KEY,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,ATR,PREV_CANDLE_COLOR,SL_SIZE,TP_SIZE
2436,230123_90000,2023-01-23,90000,69498,69500,69365,69400,7161,1266.82,1.0,428.0,857.0
2437,230123_90500,2023-01-23,90500,69400,69430,69388,69388,3497,1266.82,1.0,428.0,857.0
2438,230123_91000,2023-01-23,91000,69393,69400,69340,69340,3872,1266.82,1.0,428.0,857.0
2439,230123_91500,2023-01-23,91500,69340,69357,69204,69224,9355,1266.82,1.0,428.0,857.0
2440,230123_92000,2023-01-23,92000,69224,69246,69209,69210,6158,1266.82,1.0,428.0,857.0


In [17]:
dates = df_min["DATE"].unique()
entry_time_id = 0

for d in tqdm(dates):
    # минимальное время в дату
    min_time = df_min.loc[df_min["DATE"] == d, "TIME"].min()
    # считаем построчно срабатывание TP ил SL лонга
    df_day = df_min.loc[df_min["DATE"] == d].copy()
    for i in range(0, len(df_day)):
        entry_price = np.array(df_day["OPEN"])[entry_time_id]
        tp_long = entry_price + np.array(df_day["TP_SIZE"])[0]
        sl_long = entry_price - np.array(df_day["SL_SIZE"])[0]
        tp_short = entry_price - np.array(df_day["TP_SIZE"])[0]
        sl_short = entry_price + np.array(df_day["TP_SIZE"])[0]

        if np.array(df_day["PREV_CANDLE_COLOR"])[0] == 1:  # лонг

            if np.array(df_day["HIGH"])[i] >= tp_long:
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_LONG"] = np.array(df_day["TP_SIZE"])[0]
            elif np.array(df_day["LOW"])[i] <= sl_long:
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_LONG"] = np.array(df_day["SL_SIZE"])[0]

            # если не сработали триггеры
            else:
                profit = np.array(df_day.loc[df_day["TIME"] == 230000, "CLOSE"])[0] - entry_price
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_LONG"] = profit
        else:
            # то же самое по шорту
            if np.array(df_day["LOW"])[i] <= tp_short:
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_SHORT"] = np.array(df_day["TP_SIZE"])[0]
            elif np.array(df_day["HIGH"])[i] >= sl_short:
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_SHORT"] = np.array(df_day["SL_SIZE"])[0]
            # если не сработали триггеры
            else:
                profit = entry_price - np.array(df_day.loc[df_day["TIME"] == 230000, "CLOSE"])[0]
                df_daily.loc[df_daily["DATE"] == d, "PROFIT_SHORT"] = profit

100%|██████████| 430/430 [00:39<00:00, 10.96it/s]


In [18]:
df_daily

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,ATR,CANDLE_COLOR,PREV_CANDLE_COLOR,SL_SIZE,TP_SIZE,PROFIT_LONG,PROFIT_SHORT
14,2023-01-23,0,69498,69945,68921,69799,967899,1266.82,1,1.0,428.0,857.0,688,0
15,2023-01-24,0,69790,69791,69364,69570,658238,1207.41,0,1.0,422.0,845.0,111,0
16,2023-01-25,0,69570,69987,69421,69857,767748,1161.59,1,0.0,402.0,805.0,0,-320
17,2023-01-26,0,69857,70266,69276,69670,1189573,1149.33,0,1.0,387.0,774.0,387,0
18,2023-01-27,0,69675,70175,69649,70050,821428,1104.81,1,0.0,383.0,766.0,0,-157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,2024-09-24,0,92531,92850,91545,91583,557165,1117.29,0,0.0,368.0,735.0,0,735
440,2024-09-25,0,91618,92099,91160,91858,356389,1104.56,1,0.0,372.0,745.0,0,-414
441,2024-09-26,0,91839,92404,91609,92258,387517,1082.45,1,1.0,368.0,736.0,147,0
442,2024-09-27,0,92258,92800,92200,92740,375520,1047.99,1,1.0,361.0,722.0,269,0


In [19]:
df_daily["TOTAL_PROFIT"] = df_daily["PROFIT_LONG"] + df_daily["PROFIT_SHORT"]

In [20]:
(df_daily["TOTAL_PROFIT"].sum()) - (10 * len(df_daily))
# 78.3% по предыдущей свече. +235186
# 147364
# 400-800 - 143745

82685

In [21]:
len(df_daily.loc[df_daily["TOTAL_PROFIT"] > 0]) / len(df_daily)

0.6790697674418604

In [22]:
"""
ID | TIME  | PROFIT | WINRATE
-----------------------------
0  | 90000  | 98293 | 70.5%
-----------------------------
1  | 90500  | 85733 | 69.3%
-----------------------------
2  | 91000  | 84756 | 68.8%
-----------------------------
3  | 91500  | 83067 | 67.4%
-----------------------------
4  | 92000  | 92632 | 70.2%
-----------------------------
5  | 92500  | 82453 | 67.2%
-----------------------------
6  | 93000  | 81993 | 67.2%
-----------------------------
7  | 93500  | 87747 | 69.0%
-----------------------------
8  | 94000  | 88906 | 69.3%
-----------------------------
9  | 94500  | 89717 | 68.8%
-----------------------------
10 | 95000  | 85464 | 68.6%
-----------------------------
11 | 95500  | 84309 | 68.1%
-----------------------------
12 | 100000 | 83064 | 68.4%
-----------------------------
13 | 100500 | 85362 | 69.0%
-----------------------------
14 | 101000 | 86164 | 68.1%
-----------------------------
15 | 101500 | 89584 | 70.0%
-----------------------------
16 | 102000 | 90173 | 70.2%
-----------------------------
17 | 102500 | 87869 | 69.8%
-----------------------------
18 | 103000 | 84398 | 67.7%
-----------------------------
19 | 103500 | 81340 | 67.4%
-----------------------------
20 | 104000 | 82685 | 67.9%
-----------------------------
"""

'\nID | TIME  | PROFIT | WINRATE\n-----------------------------\n0  | 90000  | 98293 | 70.5%\n-----------------------------\n1  | 90500  | 85733 | 69.3%\n-----------------------------\n2  | 91000  | 84756 | 68.8%\n-----------------------------\n3  | 91500  | 83067 | 67.4%\n-----------------------------\n4  | 92000  | 92632 | 70.2%\n-----------------------------\n5  | 92500  | 82453 | 67.2%\n-----------------------------\n6  | 93000  | 81993 | 67.2%\n-----------------------------\n7  | 93500  | 87747 | 69.0%\n-----------------------------\n8  | 94000  | 88906 | 69.3%\n-----------------------------\n9  | 94500  | 89717 | 68.8%\n-----------------------------\n10 | 95000  | 85464 | 68.6%\n-----------------------------\n11 | 95500  | 84309 | 68.1%\n-----------------------------\n12 | 100000 | 83064 | 68.4%\n-----------------------------\n13 | 100500 | 85362 | 69.0%\n-----------------------------\n14 | 101000 | 86164 | 68.1%\n-----------------------------\n15 | 101500 | 89584 | 70.0%\n------

In [23]:
df_daily.to_excel("results_by_follow.xlsx")